
# Multi Layer Perceptron

## Import modules

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

## Load datasets

- kc_house_data: 미국 워싱턴주 시애틀(South King County)을 포함한 킹카운티(King County) 지역의 주택 가격 데이터를 포함한 공개 데이터셋
1. id: 각 주택 거래에 대한 고유 식별자.
2. date: 주택이 판매된 날짜.
3. price: 주택의 판매 가격 (종속 변수, 목표 값).
4. bedrooms: 침실의 개수.
5. bathrooms: 욕실의 개수 (부분 욕실도 포함).
6. sqft_living: 주택의 실내 면적 (평방 피트).
7. sqft_lot: 주택의 대지 면적 (평방 피트).
8. floors: 주택의 층수.
9. waterfront: 주택이 해안가에 위치해 있는지 여부 (1: 해안가, 0: 해안가 아님).
10. view: 주택에서의 전망의 질을 나타내는 지표 (0~4).
11. condition: 주택의 전반적인 상태 (1~5).
12. grade: 주택의 건축 품질과 디자인 등급 (1~13).
13. sqft_above: 지상 층의 면적 (평방 피트).
14. sqft_basement: 지하 층의 면적 (평방 피트).
15. yr_built: 주택이 지어진 연도.
16. yr_renovated: 주택이 개보수된 연도.
17. zipcode: 우편번호.
18. lat: 주택의 위도.
19. long: 주택의 경도.
20. sqft_living15: 2015년 기준으로, 인근 15개의 주택의 평균 실내 면적 (평방 피트).
21. sqft_lot15: 2015년 기준으로, 인근 15개의 주택의 평균 대지 면적 (평방 피트).

In [ ]:
# Dataset 다운로드
!wget "https://dongaackr-my.sharepoint.com/:x:/g/personal/sjkim_donga_ac_kr/ET2udlQfsxRAsvnlEtgzfi0B3HAMAmqP_Y2WRsbYrTvYaA?e=t6809f&download=1" -O kc_house_data.csv

In [ ]:
# Load dataset file
data = pd.read_csv('kc_house_data.csv')
data

## Dataset 전처리

In [ ]:
# 예측 수행에 불필요한 열 삭제
new_data = data.drop(['id', 'date'], axis=1)

# 값이 존재하지 않는 행 삭제
new_data = new_data.dropna()

# 데이터셋 가우시안 정규화
data_normalized = (new_data - new_data.mean()) / new_data.std()
data_normalized

## Dataset 특성 파악

In [ ]:
# 각 데이터에 대한 시각화
fixed_column = 'price'
used_columns = data_normalized.columns.drop(fixed_column)

num_vars = len(used_columns)
rows = (num_vars + 2) // 3  # 3열 기준으로 필요한 행 계산

fig, axs = plt.subplots(rows, 3, figsize=(15, rows * 5))

# 각 열에 대해 scatter plot을 생성하는 반복문
for i, used_column in enumerate(used_columns):
    row = i // 3
    col = i % 3
    axs[row, col].scatter(data_normalized[used_column], data_normalized[fixed_column], color='blue')
    axs[row, col].set_title(f'{used_column} vs {fixed_column}')
    axs[row, col].set_xlabel(used_column)
    axs[row, col].set_ylabel(fixed_column)

# 남은 빈 subplot 제거
for i in range(num_vars, rows * 3):
    fig.delaxes(axs.flatten()[i])

plt.tight_layout()
plt.show()

In [ ]:
# 데이터의 각 열에 대한 상관관계 확인
correlation_matrix = data_normalized.corr()
correlation_matrix

## 집 가격 예측에 사용할 데이터 지정


In [ ]:
# 예측에 사용할 데이터들에 대한 2차원 행렬 변환
x = np.array(data_normalized[['sqft_living']])
y = np.array(data_normalized[['price']])

# Train dataset / Test dataset 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

# Train dataset 형상 확인
print(x_train.shape)
print(y_train.shape)

## Single Layer Perceptron

In [ ]:
class SLP(nn.Module):
  def __init__(self):
    super(SLP, self).__init__()

    self.fc1 = nn.Linear(1, 1, bias=True)

  def forward(self, x):
    y = self.fc1(x)

    return y

## Hyper-parameters 지정

In [ ]:
batch_size = 100
learning_rate = 0.01
training_epochs = 50
loss_function = nn.MSELoss()
network = SLP()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

## Perceptron 학습을 위한 반복문 선언

In [ ]:
network.train()

for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(x_train) // batch_size

  for batch in range(total_batch):
    input = torch.tensor(x_train[batch * batch_size : (batch * batch_size) + batch_size, :], dtype=torch.float32)
    label = torch.tensor(y_train[batch * batch_size : (batch * batch_size) + batch_size, :], dtype=torch.float32)

    pred = network(input)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

In [ ]:
for param in network.parameters():
    print(param)

## Prediction

In [ ]:
# Test dataset을 이용한 예측
input = torch.tensor(x_test, dtype=torch.float32)
network.eval()
with torch.no_grad(): # test에서는 기울기 계산 제외
    y_hat = network(input)
y_hat = y_hat.detach().numpy()

In [ ]:
# 정답 데이터와 예측 데이터 간 차이 계산
def MSE (pred, label):
  error = pred - label
  mse = np.mean(error ** 2)
  return mse

print(MSE(y_hat, y_test))

In [ ]:
# 시각화
fig = plt.figure(figsize=(8,6))
plt.scatter(x_test, y_test, color='b', marker='o', s=30)
plt.plot(x_test, y_hat, color='r')
plt.show()